# 1. Load data

In [272]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [273]:
cn = pd.read_csv("./data/concentration/2020_20230814.csv")
q = pd.read_csv("./data/quote/2020_20230814.csv")

In [274]:
merged_df = pd.merge(q, cn, on=['日期', '股號'], how='left')

# 2. Clean data

## 2.1 CN
- 沒有NA
- 有0但應該沒關係

In [275]:
# check na 
na_index_cn  = cn[cn.isnull().any(axis=1)].index
cn.iloc[na_index_cn]

,日期,股號,主力買賣超,散戶買賣超,5日籌碼集中度


In [276]:
# check 0
zero_index = (cn==0).any(axis=1)
cn.loc[zero_index]

,日期,股號,主力買賣超,散戶買賣超,5日籌碼集中度
3,2023-01-03,8099,0,-4,11.68
5,2023-01-05,8099,0,-3,9.76
6,2023-01-06,8099,0,-28,8.94
10,2023-01-12,8099,0,-2,8.72
12,2023-01-16,8099,0,0,13.54
...,...,...,...,...,...
1312635,2022-08-02,4956,0,25,-3.89
1312640,2022-08-09,4956,-4,0,5.78
1312670,2022-04-20,4956,1,0,-4.87
1312679,2022-03-31,4956,23,0,16.92


## 2.2 q

- drop NA and 0 value

In [277]:
# detect NA
na_index_q  = q[q.isnull().any(axis=1)].index
q.iloc[na_index_q]

# drop NA
q = q.dropna()

# 3. Merge df

In [278]:
df = pd.merge(q, cn, on=['股號', '日期'], how='left').dropna()

df = df.groupby('股號').apply(lambda x: x.sort_values('日期')).reset_index(drop=True)
# 選擇一隻成交量>500的
# print(df[df.groupby('股號')['成交量'].transform('mean') > 500])
# df_4956 = df[df['股號']==4956]

# 4. 量起

## 4.1 成交量倍增且前一期不低於500

In [279]:
df['成交量_1'] = df.groupby('股號')['成交量'].shift(-1)

def divide_two_cols(df_sub):
    df_sub['volume_delta_1'] = df_sub['成交量_1'] / df_sub['成交量']
    return df_sub

df = df.groupby('股號').apply(divide_two_cols)

In [280]:
df_vol_increse = df[(df['成交量']>500)&(df['volume_delta_1']>=2)]

In [281]:
# df_vol_increse[df_vol_increse['股號']==1101]

# 價揚

In [289]:
df_vol_increse['ret'] = df_vol_increse.groupby('股號')['收盤價'].pct_change()
df_vol_increse = df_vol_increse.dropna()

In [290]:
df_vol_increse['股號']

,日期,股號,收盤價,漲跌幅(%),成交量,成交值(百萬),主力買賣超,散戶買賣超,5日籌碼集中度,成交量_1,volume_delta_1,ret
44,2020-03-16,1101,38.85,-3.48,20548,811.67,-1389.0,6983.0,-3.39,43649.0,2.124246,-0.135706
62,2020-04-13,1101,40.80,-0.49,5565,227.71,-170.0,686.0,-3.04,15494.0,2.784187,0.050193
86,2020-05-18,1101,42.35,0.12,10674,452.03,56.0,495.0,-8.15,21443.0,2.008900,0.037990
94,2020-05-28,1101,42.35,-1.17,16239,691.42,-6220.0,16742.0,-18.37,37201.0,2.290843,0.000000
109,2020-06-18,1101,42.70,-0.35,10558,450.94,-2591.0,7001.0,-20.92,25125.0,2.379712,0.008264
...,...,...,...,...,...,...,...,...,...,...,...,...
1181260,2021-10-05,912000,2.69,0.00,638,1.71,40.0,18.0,28.20,1276.0,2.000000,0.003731
1181303,2021-12-06,912000,2.80,1.08,527,1.47,138.0,-187.0,29.02,1631.0,3.094877,0.040892
1181407,2022-05-17,912000,2.18,9.55,506,1.06,28.0,25.0,1.01,1040.0,2.055336,-0.221429
1181514,2022-10-18,912000,2.27,-0.44,645,1.47,9.0,-113.0,14.50,1905.0,2.953488,0.041284
